<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Extraction" data-toc-modified-id="Extraction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Extraction</a></span></li><li><span><a href="#Loading" data-toc-modified-id="Loading-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Loading</a></span></li><li><span><a href="#Splitting" data-toc-modified-id="Splitting-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Splitting</a></span></li><li><span><a href="#Storing" data-toc-modified-id="Storing-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Storing</a></span></li><li><span><a href="#Retrieval" data-toc-modified-id="Retrieval-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Retrieval</a></span></li><li><span><a href="#Question-Answering" data-toc-modified-id="Question-Answering-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Question Answering</a></span></li></ul></div>

In [1]:
# !pip install langchain openai chromadb python-dotenv

In [2]:
# Set env var OPENAI_API_KEY or load from a .env file
#import dotenv
#dotenv.load_dotenv()

In [3]:
# main.py
import json
import os
import telebot

# Загрузка конфигурации из JSON-файла
with open('config.json') as config_file:
    config = json.load(config_file)

# Установка переменных окружения
os.environ["LANGCHAIN_API_KEY"] = config["LANGCHAIN_API_KEY"]
os.environ["OPENAI_API_KEY"] = config["OPENAI_API_KEY"]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]="nlmk_bot"

# Extraction

In [4]:
from extract_urls import get_hyperlinks
from pprint import pprint

In [5]:
import requests
from bs4 import BeautifulSoup

def get_href_list():
    url = "https://evrazsteel.ru/publication/news/"
    
    response = requests.get(url)
    response.raise_for_status()  # Проверяем, успешен ли запрос
    
    soup = BeautifulSoup(response.text, 'html.parser')
    a_tags = soup.find_all('a', class_='card__title')
    href_list = ['https://evrazsteel.ru' + a['href'] for a in a_tags if a.has_attr('href')]
    
    return href_list

# Пример использования:
urls = get_href_list()
print(urls)


['https://evrazsteel.ru/publication/news/parking-gk-ferro-stroy-v-aeroportu-vnukovo', 'https://evrazsteel.ru/publication/news/edinaya-metodika-ocenki-primeneniya-ii-v-stroitelstve', 'https://evrazsteel.ru/publication/news/metodika-opredeleniya-stoimosti-informacionnoy-modeli-v-stroyke', 'https://evrazsteel.ru/publication/news/5-svodov-pravil-dlya-stalnogo-stroitelstva-v-rossii', 'https://evrazsteel.ru/publication/news/evraz-i-kgasu-podpisali-soglashenie-o-sotrudnichestve', 'https://evrazsteel.ru/publication/news/kamchatka-metodika-issledovaniya-seysmostoykosti-zdaniy', 'https://evrazsteel.ru/publication/news/bezopasnost-stroitelnyh-materialov-i-izdeliy', 'https://evrazsteel.ru/publication/news/gost-zashchita-ot-korrozii', 'https://evrazsteel.ru/publication/news/18-novyh-kompaniy-arss']


# Loading

In [6]:
!pip show aiohttp

Name: aiohttp
Version: 3.9.3
Summary: Async http client/server framework (asyncio)
Home-page: https://github.com/aio-libs/aiohttp
Author: 
Author-email: 
License: Apache 2
Location: /home/fedor/miniconda3/envs/jupyterlab/lib/python3.9/site-packages
Requires: aiosignal, async-timeout, attrs, frozenlist, multidict, yarl
Required-by: e2b, langchain, langchain-community, langchain-fireworks


In [7]:
from langchain.document_loaders import WebBaseLoader

In [8]:
%%time
loader = WebBaseLoader(urls)

CPU times: user 891 µs, sys: 435 µs, total: 1.33 ms
Wall time: 3.76 ms


In [9]:
data = loader.load()

In [10]:
#print(data[0].page_content)

# Splitting

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [12]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)

In [13]:
all_splits = text_splitter.split_documents(data)

In [14]:
len(all_splits)

50

In [15]:
print(all_splits[10].page_content)

17 августа 2023

Программа сотрудничества для партнёров EVRAZ STEEL BOX





27 АПРЕЛЯ 2024
нормы
строительство







Еще по теме


26 апреля 2024



Модельный подход

                    Власти утвердили методику определения стоимости информационной модели в стройке.                    

нормы



25 марта 2024



Стальная сотня

                    К партнёрской программе EVRAZ STEEL BUILDING присоединились уже 100 заводов металлоконструкций – каждое пятое от всех конкурентоспособных предприятий отрасли в России.                    

партнёрство
производство
металлоконструкции



22 февраля 2024



Новости стального строительства: февраль 2024

                    Рассказываем об интересных событиях и проектах отрасли стального строительства за месяц.                    

строительство
отрасль
монтаж



21 июля 2023


# Storing 

In [16]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
import os


In [17]:
import shutil

# Путь к директории, которую вы хотите удалить
directory_path = 'langchain_embeddings'

# Удаление директории и всех поддиректорий
try:
    shutil.rmtree(directory_path)
    print(f"Директория {directory_path} успешно удалена")
except Exception as e:
    print(f"Ошибка при удалении директории {directory_path}: {e}")


Директория langchain_embeddings успешно удалена


In [18]:
vectorstore = Chroma.from_documents(documents=all_splits, 
                                    embedding=OpenAIEmbeddings(), 
                                    persist_directory='langchain_embeddings')

/home/fedor/miniconda3/envs/jupyterlab/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [19]:
vectorstore._collection.count()

50

# Retrieval

In [20]:
from langchain.chat_models import ChatOpenAI

In [21]:
#llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
#llm = ChatOpenAI(model_name="gpt-4-turbo-preview", temperature=0)
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

/home/fedor/miniconda3/envs/jupyterlab/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [22]:
from langchain.chains import RetrievalQA

In [23]:
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectorstore.as_retriever(search_kwargs={'k':10}),
                                       return_source_documents=True,
                                       )

# Question Answering

In [24]:
question = "что нового в евразе?"

In [25]:
result = qa_chain({"query": question})

/home/fedor/miniconda3/envs/jupyterlab/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [26]:
print(result['result'])

Вот несколько последних новостей о ЕВРАЗе:

1. **28 ноября 2023** - ЕВРАЗ поставит металлоконструкции для строительства учебно-выставочного комплекса аграрного университета им. Тимирязева в Москве.

2. **15 января 2024** - Мосгосстройнадзор выдал заключение о соответствии на многоуровневый паркинг на территории жилого квартала «Перовское 2».

3. **01 февраля 2024** - В январском выпуске журнала «Металлоснабжение и сбыт» опубликованы рейтинги, в которые вошли ЕВРАЗ и ЕВРАЗ Маркет.

Если вам нужны более подробные сведения, пожалуйста, уточните ваш запрос.


In [ ]:
import telebot
import time

bot = telebot.TeleBot(config["TELEGRAM_BOT_TOKEN"])


@bot.message_handler(content_types=['text'])
def handle_message(message):
    # Проверка на упоминание бота в сообщении
    if '@didgest_nlmk_bot' in message.text:
        
        question = message.text.replace('@didgest_nlmk_bot', '').strip()  # Удаляем упоминание бота из текста вопроса
        bot.send_message(message.chat.id, 'Надо подумать...')
        result = qa_chain({"query": question + " Не отвечай на вопросы не связанные с дайджестом новостей. Проси задать тебе вопросы только по дайцджесту новостей."})
        bot.send_message(message.chat.id, result['result'])

while True:
    try:
        bot.polling(none_stop=True)
    except:
        print('Error occurred, retrying...')
        time.sleep(60)


2024-05-26 23:07:44,824 (__init__.py:1147 MainThread) ERROR - TeleBot: "Threaded polling exception: A request to the Telegram API was unsuccessful. Error code: 502. Description: Bad Gateway"
2024-05-26 23:07:44,831 (__init__.py:1149 MainThread) ERROR - TeleBot: "Exception traceback:
Traceback (most recent call last):
  File "/home/fedor/miniconda3/envs/jupyterlab/lib/python3.9/site-packages/telebot/__init__.py", line 1140, in __threaded_polling
    polling_thread.raise_exceptions()
  File "/home/fedor/miniconda3/envs/jupyterlab/lib/python3.9/site-packages/telebot/util.py", line 110, in raise_exceptions
    raise self.exception_info
  File "/home/fedor/miniconda3/envs/jupyterlab/lib/python3.9/site-packages/telebot/util.py", line 92, in run
    task(*args, **kwargs)
  File "/home/fedor/miniconda3/envs/jupyterlab/lib/python3.9/site-packages/telebot/__init__.py", line 661, in __retrieve_updates
    updates = self.get_updates(offset=(self.last_update_id + 1),
  File "/home/fedor/miniconda3/

Error occurred, retrying...
Error occurred, retrying...
Error occurred, retrying...


2024-05-27 22:06:51,957 (__init__.py:1147 MainThread) ERROR - TeleBot: "Threaded polling exception: A request to the Telegram API was unsuccessful. Error code: 502. Description: Bad Gateway"
2024-05-27 22:06:52,053 (__init__.py:1149 MainThread) ERROR - TeleBot: "Exception traceback:
Traceback (most recent call last):
  File "/home/fedor/miniconda3/envs/jupyterlab/lib/python3.9/site-packages/telebot/__init__.py", line 1140, in __threaded_polling
    polling_thread.raise_exceptions()
  File "/home/fedor/miniconda3/envs/jupyterlab/lib/python3.9/site-packages/telebot/util.py", line 110, in raise_exceptions
    raise self.exception_info
  File "/home/fedor/miniconda3/envs/jupyterlab/lib/python3.9/site-packages/telebot/util.py", line 92, in run
    task(*args, **kwargs)
  File "/home/fedor/miniconda3/envs/jupyterlab/lib/python3.9/site-packages/telebot/__init__.py", line 661, in __retrieve_updates
    updates = self.get_updates(offset=(self.last_update_id + 1),
  File "/home/fedor/miniconda3/

Error occurred, retrying...
Error occurred, retrying...
